In [2]:
import numpy as np
import copy
import math

In [10]:
def make_cheb(order, Tn, Tn_1, step, x):
    step = step + 1
    
    if step == 0:
        T = 1.0
    
    if step == 1:
        T = float(x)
    
    if step > 1:
        T = 2 * x * Tn - Tn_1

    if step == order:
        return T
    else:
        return make_cheb(order, T, Tn, step, x)
        

def make_n_collection(num):
    n_collect = np.array([])
    
    n_collect = np.append(n_collect, -(num // 2))
    for i in range(1, num):
        n_collect = np.append(n_collect, n_collect[i - 1] + 1)
    return n_collect

def x_make_function(k, num):
    return(math.cos((math.pi * (1 + 2 * k)) / (2 * (num + 1))))
    

def real_function(x):
    return x ** 3
        
def solve_system(shape, x, y):
    
    system = np.zeros(shape)
    F = np.zeros((shape[0], 1))

    for i in range(shape[0]):
        b = np.vectorize(make_cheb)(i, x, 1, -1, x)
        F[i] = sum(b * y)
        for j in range(shape[0]):
            a = np.vectorize(make_cheb)(j, x, 1, -1, x)
            system[i][j] = sum(a * b)
    
    solution = np.linalg.solve(system, F)
    #print(system)
    #print("\n\n",F)
    return solution

def print_coef(sol):
    for i in range(sol.shape[0]):
        print(f"C{i} = ", {sol[i][0]})
        
def find_aprox_mean(x, C_arr):
    T = np.zeros((x.shape[0], C_arr.shape[0]))

    for i in range(T.shape[0]):
        x0 = x[i]
        for j in range(T.shape[1]):
            T[i][j] = make_cheb(j, x0, 1, -1, x0)
    
    res = np.dot(T, C_arr)
    res = np.reshape(res, res.shape[0])
    return res

def find_min_delta(x, y, max_order):
    delta_min = 1000
    num_aprox = 0
    for n in range(2, max_order + 1):
        delta_max = 0
        
        sol = solve_system((n, n), x, y)
        aprox_mean = find_aprox_mean(x, sol)
        
        delta = y - aprox_mean
        for i in range(len(delta)):
            if abs(delta[i]) > delta_max:
                delta_max = abs(delta[i])
        
        if delta_max < delta_min:
            delta_min = delta_max
            num_aprox = n
        print("Порядок ", n)
        print("Значания многочлена P(x)", aprox_mean)
        print("Значания функции", y)
        print("Значения дельта", delta)
        print("Максимальная дельта", delta_max)
        print("Наилучшее отклонение", delta_min)
        print("\n")
        
        
    print("_________________________")
    print("Наилучшая степень многочлена Чебышева", num_aprox)
    print("Наилучшее отклонение", delta_min)

In [11]:
cheb = make_cheb(6, 2, 1, -1, 2)

n = make_n_collection(10)

x = np.vectorize(x_make_function)(n, len(n))
y = np.vectorize(real_function)(x)

print(x)
print(y)

[0.28173256 0.54064082 0.75574957 0.909632   0.98982144 0.98982144
 0.909632   0.75574957 0.54064082 0.28173256]
[0.02236202 0.15802525 0.43165198 0.75265714 0.96977408 0.96977408
 0.75265714 0.43165198 0.15802525 0.02236202]


In [12]:
T = np.vectorize(make_cheb)(0, x, 1, -1, x)
print(T)
print(sum(T))

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
10.0


In [13]:
order = 3
sol = solve_system((order, order), x, y)

In [14]:
print_coef(sol)

C0 =  {1.1394410678223736}
C1 =  {-1.1041108520806333}
C2 =  {0.9537899130527374}


In [15]:
collection = np.array([0.5, 0.1])
res = find_aprox_mean(collection, sol)
print(collection)
print(res)

[0.5 0.1]
[0.11049069 0.09431587]


In [18]:
find_min_delta(x, y, 10)

Порядок  2
Значания многочлена P(x) [-0.08121509  0.26174267  0.54668228  0.75051962  0.856741    0.856741
  0.75051962  0.54668228  0.26174267 -0.08121509]
Значания функции [0.02236202 0.15802525 0.43165198 0.75265714 0.96977408 0.96977408
 0.75265714 0.43165198 0.15802525 0.02236202]
Значения дельта [ 0.10357712 -0.10371742 -0.1150303   0.00213752  0.11303308  0.11303308
  0.00213752 -0.1150303  -0.10371742  0.10357712]
Максимальная дельта 0.11503030094840694
Наилучшее отклонение 0.11503030094840694


Порядок  3
Значания многочлена P(x) [0.02599796 0.14629503 0.44074822 0.75970607 0.96172319 0.96172319
 0.75970607 0.44074822 0.14629503 0.02599796]
Значания функции [0.02236202 0.15802525 0.43165198 0.75265714 0.96977408 0.96977408
 0.75265714 0.43165198 0.15802525 0.02236202]
Значения дельта [-0.00363594  0.01173023 -0.00909624 -0.00704894  0.00805089  0.00805089
 -0.00704894 -0.00909624  0.01173023 -0.00363594]
Максимальная дельта 0.011730227561432355
Наилучшее отклонение 0.011730227